# Web Application Development: FastAPI Backend and Streamlit UI
* FastAPI Backend

In [ ]:
from fastapi import FastAPI, File, UploadFile
import uvicorn
import cv2

app = FastAPI()

# Load best model (assuming nested_unet was saved)
nested_unet.load_weights('nested_unet_weights.h5')

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    img = cv2.imdecode(np.frombuffer(contents, np.uint8), cv2.IMREAD_UNCHANGED)
    img = preprocess_image(img)
    img = np.expand_dims(img, axis=[0, -1])  # Shape: (1, 256, 256, 1)

    # Predict and return segmentation result
    prediction = nested_unet.predict(img)
    return {"prediction": prediction.tolist()}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

* Streamlit UI

In [ ]:
import streamlit as st
import requests
import numpy as np
from PIL import Image

st.title("Brain MRI Metastasis Segmentation")

uploaded_file = st.file_uploader("Upload a Brain MRI Image", type="png")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Send the image to the FastAPI backend
    files = {'file': uploaded_file.getvalue()}
    response = requests.post("http://localhost:8000/predict/", files=files)

    # Display segmentation results
    if response.status_code == 200:
        prediction = np.array(response.json()['prediction'])
        st.image(prediction.squeeze(), caption='Segmented Metastasis', use_column_width=True)